In [1]:
import os
import pandas as pd
import pandasql as ps

In [35]:
path = ''
df = pd.read_csv('/Users/niyubahweany-arlene/git/ijambo-data/ijambo-data.csv', parse_dates=[10,11,12,13,14,15])
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3481 entries, 0 to 3480
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   country               3481 non-null   object             
 1   guesses               3481 non-null   object             
 2   id                    3481 non-null   int64              
 3   score                 3481 non-null   float64            
 4   solution              3481 non-null   object             
 5   time_taken            3481 non-null   int64              
 6   timezone              3448 non-null   object             
 7   won                   3481 non-null   bool               
 8   start_time            3481 non-null   object             
 9   end_time              3481 non-null   object             
 10  converted_start_time  3481 non-null   datetime64[ns, UTC]
 11  start_date            3481 non-null   datetime64[ns]     
 12  start_

In [3]:
# players per day
q1 = """SELECT start_date, c AS count_players,

SUM(c) OVER(ORDER BY start_date) AS running_total_players
FROM (
  SELECT start_date, COUNT(DISTINCT id) c
  FROM (
    SELECT MIN(start_date) start_date, id
    FROM df
    GROUP BY id
  ) 
  GROUP BY start_date
) 
"""

print(ps.sqldf(q1, locals()))

                   start_date  count_players  running_total_players
0  1970-01-01 00:00:00.000000             46                     46
1  2022-03-29 00:00:00.000000              1                     47
2  2022-03-30 00:00:00.000000             81                    128
3  2022-03-31 00:00:00.000000            723                    851
4  2022-04-01 00:00:00.000000            652                   1503
5  2022-04-02 00:00:00.000000            715                   2218
6  2022-04-03 00:00:00.000000            731                   2949
7  2022-04-04 00:00:00.000000            532                   3481


In [25]:
# players per country
q2 = """ SELECT
    DISTINCT country,
    COUNT (id) AS count_players
FROM
    df
GROUP BY
    country

ORDER BY COUNT (id) DESC

LIMIT 10    
"""

print(ps.sqldf(q2, locals()))

          country  count_players
0         Burundi           1736
1          Canada            467
2          France            429
3          Rwanda            221
4   United States            151
5           Kenya             93
6  United Kingdom             47
7         Belgium             43
8     South Korea             33
9          Sweden             31


In [30]:
# players per country
q3 = """ SELECT
    DISTINCT country,
    start_date,
    COUNT (id) AS count_players
FROM
    df
GROUP BY
    country, start_date

ORDER BY start_date DESC, COUNT (id) DESC

LIMIT 10    
"""

print(ps.sqldf(q3, locals()))

          country                  start_date  count_players
0         Burundi  2022-04-04 00:00:00.000000            288
1          Canada  2022-04-04 00:00:00.000000             58
2          France  2022-04-04 00:00:00.000000             51
3          Rwanda  2022-04-04 00:00:00.000000             34
4           Kenya  2022-04-04 00:00:00.000000             20
5   United States  2022-04-04 00:00:00.000000             18
6  United Kingdom  2022-04-04 00:00:00.000000              8
7          Sweden  2022-04-04 00:00:00.000000              7
8     South Korea  2022-04-04 00:00:00.000000              6
9         Belgium  2022-04-04 00:00:00.000000              6


In [41]:
# rank score by country 
q4 = """ SELECT
    DISTINCT country,
    avg(score) AS avg_score 
FROM
    df
WHERE won = 1
GROUP BY
    country
ORDER BY avg(score) DESC

LIMIT 10    
"""

print(ps.sqldf(q4, locals()))

         country     avg_score
0      Australia  4.122333e+09
1        Denmark  1.236628e+09
2        Belgium  2.498345e+08
3         Canada  1.530792e+08
4  United States  1.417060e+08
5         Rwanda  1.276054e+08
6         France  6.890189e+07
7        Burundi  4.162122e+07
8         Norway  8.291833e+04
9         Israel  3.716625e+04


In [42]:
#rank score by country per day
q5 = """SELECT
    country,
    start_date,
    avg(score)  AS avg_score
from
    (
        select *,avg(score) OVER (PARTITION BY country, start_date) 
        from df
    ) 
WHERE won = 1
GROUP BY
     country, start_date
ORDER BY
    start_date DESC, avg(score) DESC

LIMIT 10
"""

print(ps.sqldf(q5, locals()))

          country                  start_date      avg_score
0          Norway  2022-04-04 00:00:00.000000  104312.000000
1     South Korea  2022-04-04 00:00:00.000000   90227.000000
2  United Kingdom  2022-04-04 00:00:00.000000   21886.000000
3       Singapore  2022-04-04 00:00:00.000000   13990.000000
4         Burundi  2022-04-04 00:00:00.000000   12346.587678
5     Netherlands  2022-04-04 00:00:00.000000   11206.750000
6          France  2022-04-04 00:00:00.000000   10792.609756
7        Tanzania  2022-04-04 00:00:00.000000   10551.000000
8           China  2022-04-04 00:00:00.000000   10357.500000
9          Canada  2022-04-04 00:00:00.000000    6506.239130
